In [1]:
from tensorflow import keras
size = 224
base_model = keras.applications.VGG19(weights='imagenet', input_shape=(size, size, 3), include_top=False)
base_model.trainable = False

inputs = keras.Input(shape=(size, size, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
                    
model.summary()

80150528/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 20,024,897
Trainable params: 513
Non-trainable params: 20,024,384
_________________________________________________________________


In [2]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(samplewise_center=True,
                rotation_range=10,
                zoom_range=0.2,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True,
                vertical_flip=True)

In [ ]:
train_it = datagen.flow_from_directory('train',
                     target_size=(224, 224),
                     color_mode='rgb',
                     batch_size=6)

valid_it = datagen.flow_from_directory('valid',
                     target_size=(224, 224),
                     color_mode='rgb',
                     batch_size=6)

In [ ]:
model.fit(train_it,steps_per_epoch=train_it.samples/train_it.batch_size,
     validation_data=valid_it, validation_steps=valid_it.samples/valid_it.batch_size,
     epochs=20)

In [ ]:
base_model.trainable = True
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
       loss=keras.losses.BinaryCrossentropy(from_logits=True),
       metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it,steps_per_epoch=train_it.samples/train_it.batch_size,
     validation_data=valid_it, validation_steps=valid_it.samples/valid_it.batch_size,
     epochs=20)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds
    
def foreign_object_detector(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("道路順暢 沒有障礙")
    else:
        print("小心!!前方有障礙")

In [ ]:
foreign_object_detector("valid/只有路/道路照片 - Google 搜尋 5.png")